<a href="https://colab.research.google.com/github/Octaman888/Reverse-Mortgage-Predictors/blob/main/RMCapstone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [300]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [301]:
import numpy as np 
import pandas as pd #data frames
from sklearn.model_selection import train_test_split #creating train and test splits
from sklearn.metrics import confusion_matrix, classification_report # model performance for logistic
import statsmodels.api as sm #regression models
import statsmodels.formula.api as smf # This allows you to use formulas inside statmodels functions
import matplotlib.pyplot as plt
import seaborn as sns


In [302]:
loan_data = pd.read_csv("/content/drive/MyDrive/2020_bexar.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [303]:
print(loan_data.columns)


Index(['activity_year', 'lei', 'derived_msa_md', 'state_code', 'county_code',
       'census_tract', 'conforming_loan_limit', 'derived_loan_product_type',
       'derived_dwelling_category', 'derived_ethnicity', 'derived_race',
       'derived_sex', 'action_taken', 'purchaser_type', 'preapproval',
       'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage',
       'open_end_line_of_credit', 'business_or_commercial_purpose',
       'loan_amount', 'combined_loan_to_value_ratio', 'interest_rate',
       'rate_spread', 'hoepa_status', 'total_loan_costs',
       'total_points_and_fees', 'origination_charges', 'discount_points',
       'lender_credits', 'loan_term', 'prepayment_penalty_term',
       'intro_rate_period', 'negative_amortization', 'interest_only_payment',
       'balloon_payment', 'other_nonamortizing_features', 'property_value',
       'construction_method', 'occupancy_type',
       'manufactured_home_secured_property_type',
       'manufactured_home_land_property_in

In [304]:
#Transform RM column

In [305]:
loan_data_filtered = loan_data[loan_data["reverse_mortgage"] != 1111]

In [306]:
loan_data_filtered["reverse_mortgage"].value_counts()

2    143262
1       279
Name: reverse_mortgage, dtype: int64

In [307]:
loan_data_filtered.describe()

,activity_year,derived_msa_md,county_code,census_tract,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,hoepa_status,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,income,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
count,143541.0,143541.0,143541.0,1.428630e+05,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,1.435410e+05,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,1.073970e+05,143541.000000,143541.000000,143516.000000,23497.000000,404.000000,13.000000,5.000000,143533.000000,9858.000000,155.000000,3.000000,0.0,143541.000000,143541.000000,143521.000000,5004.000000,663.000000,107.000000,16.000000,143537.000000,2178.000000,268.000000,34.000000,4.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.000000,143541.00000,143541.000000,143541.000000,5023.000000,1717.000000,447.000000,385.000000,143541.000000,2978.000000,302.000000,22.000000,143541.000000,143541.000000,143541.0,143541.000000,143541.000000,143541.000000,143541.000000
mean,2020.0,41700.0,48029.0,4.802916e+10,2.708836,5.323678,1.992796,1.726378,15.904355,1.043040,1.998056,1.976028,1.976515,2.290209e+05,2.432392,2.077190,2.062080,2.069882,2.075191,1.023429,1.112330,2.969667,4.935872,8.525844e+02,5.306644,8.438105,2.210527,11.183811,12.735149,13.307692,13.600000,3.746616,11.150639,12.264516,13.666667,NaN,2.182540,3.161153,5.269926,18.388090,22.822021,26.084112,31.687500,6.792207,19.374656,24.660448,25.176471,24.750000,2.181941,3.160818,1.764214,3.692241,2.182742,3.161550,1.35695,1.448896,3.707756,2.497113,2.231217,1.091723,1.077922,9.272166,4.252183,5.271523,6.000000,6336.435032,61.667870,72200.0,130.742324,1462.690291,1907.046690,23.329766
std,0.0,0.0,0.0,2.342438e+05,1.911438,16.414279,0.084568,0.863577,14.911315,0.202948,0.044045,0.152963,0.151437,4.727449e+05,0.496028,9.256145,9.257075,9.256598,9.256269,0.151262,0.449736,0.208298,0.452885,8.452177e+03,3.534350,2.563663,1.577927,1.682759,1.806152,0.751068,0.547723,1.682936,1.622021,2.126143,0.577350,NaN,0.408758,0.926053,1.945067,10.249246,11.075339,13.744379,13.098187,1.840013,9.724035,10.955836,15.617532,16.007811,0.409348,0.926938,1.013297,1.520865,0.411140,0.926364,0.72686,0.802499,2.337698,1.706788,1.313638,0.311367,0.321383,2.175760,2.816383,2.651230,2.845213,2492.870317,18.927573,0.0,52.623434,636.757282,736.338884,16.267971
min,2020.0,41700.0,48029.0,4.802911e+10,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000e+03,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-7.840000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,12.

In [308]:
loan_data_filtered["loan_term"].value_counts()

360    94611
180    15034
360    10344
240     5961
120     2274
       ...  
556        1
492        1
322        1
73         1
548        1
Name: loan_term, Length: 559, dtype: int64

In [309]:
loan_data_filtered["loan_term"].describe()

count     142693
unique       559
top          360
freq       94611
Name: loan_term, dtype: object

In [310]:
loan_data_filtered['loan_term'] = pd.to_numeric(loan_data['loan_term'], errors = 'coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [311]:
loan_data_filtered = loan_data[loan_data["loan_term"] != "NA"]

In [312]:
loan_data_filtered = loan_data[loan_data["loan_term"] != "Exempt"]

In [313]:
loan_data_filtered["loan_term"].describe()

count     142693
unique       559
top          360
freq       94611
Name: loan_term, dtype: object

In [314]:
loan_data_filtered["loan_term"].value_counts()

360    94611
180    15034
360    10344
240     5961
120     2274
       ...  
556        1
492        1
322        1
73         1
548        1
Name: loan_term, Length: 559, dtype: int64

In [315]:
#Pull relavent available information from loan_data to Reverse Mortgages (RM) from data

In [316]:
RM = loan_data_filtered[['census_tract','reverse_mortgage','open_end_line_of_credit','loan_amount','property_value','income', 'loan_term','applicant_age','lien_status','tract_population']]
RM

,census_tract,reverse_mortgage,open_end_line_of_credit,loan_amount,property_value,income,loan_term,applicant_age,lien_status,tract_population
0,4.802917e+10,2,2,335000,425000,345.0,360,45-54,1,7751
1,4.802918e+10,2,2,325000,425000,294.0,360,35-44,1,4610
2,4.802912e+10,2,2,215000,215000,NaN,360,35-44,1,4664
3,4.802919e+10,2,2,315000,475000,225.0,360,35-44,1,11921
4,4.802916e+10,2,2,185000,185000,95.0,360,35-44,1,7623
...,...,...,...,...,...,...,...,...,...,...
145268,4.802919e+10,2,2,345000,535000,216.0,360,35-44,1,4886
145269,4.802919e+10,2,2,505000,705000,181.0,180,35-44,1,7042
145270,4.802919e+10,2,2,285000,405000,120.0,360,45-54,1,11921
145271,4.802918e+10,2,2,295000,375000,132.0,360,35-44,1,9504


In [317]:
RM.shape

(143541, 10)

In [318]:
RM["reverse_mortgage"].value_counts()

2    143262
1       279
Name: reverse_mortgage, dtype: int64

In [319]:
RM["census_tract"].value_counts()

4.802917e+10    5674
4.802912e+10    3773
4.802918e+10    3236
4.802918e+10    2878
4.802917e+10    2535
                ... 
4.802998e+10       3
4.809131e+10       2
4.802998e+10       2
4.802913e+10       1
4.802998e+10       1
Name: census_tract, Length: 366, dtype: int64

In [320]:
RM["open_end_line_of_credit"].value_counts()

2    140100
1      3441
Name: open_end_line_of_credit, dtype: int64

In [321]:
RM["loan_amount"].value_counts()

185000      6522
205000      6469
195000      6255
175000      5982
215000      5976
            ... 
3185000        1
10565000       1
2045000        1
10335000       1
1805000        1
Name: loan_amount, Length: 289, dtype: int64

In [322]:
RM["property_value"].value_counts()

215000      4531
225000      4496
205000      4477
185000      4408
195000      4295
            ... 
2545000        1
24805000       1
19585000       1
2255000        1
3255000        1
Name: property_value, Length: 449, dtype: int64

In [323]:
RM["income"].value_counts()

0.0         2630
60.0        1400
54.0        1186
65.0        1183
55.0        1181
            ... 
57899.0        1
41520.0        1
57914.0        1
166177.0       1
57000.0        1
Name: income, Length: 1868, dtype: int64

In [324]:
RM["loan_term"].value_counts()

360    94611
180    15034
360    10344
240     5961
120     2274
       ...  
556        1
492        1
322        1
73         1
548        1
Name: loan_term, Length: 559, dtype: int64

In [325]:
RM["applicant_age"].value_counts()

35-44    35650
25-34    30527
45-54    27231
55-64    18177
8888     15143
65-74     9735
<25       3915
>74       3163
Name: applicant_age, dtype: int64

In [326]:
RM["lien_status"].value_counts()

1    137363
2      6178
Name: lien_status, dtype: int64

In [327]:
RM["tract_population"].value_counts()

10857    5674
10923    3773
2385     3236
7597     2878
5554     2603
         ... 
7714        4
6824        2
520         2
1120        1
1332        1
Name: tract_population, Length: 352, dtype: int64

In [328]:
RM['property_value'].describe 

<bound method NDFrame.describe of 0         425000
1         425000
2         215000
3         475000
4         185000
           ...  
145268    535000
145269    705000
145270    405000
145271    375000
145272    375000
Name: property_value, Length: 143541, dtype: object>

In [329]:
RM['property_value'] = pd.to_numeric(RM['property_value'], errors = 'coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [330]:
RM['property_value'].mean()

303789.944319301

In [331]:
RM['property_value'].value_counts()

215000.0      4787
205000.0      4781
225000.0      4765
185000.0      4689
195000.0      4551
              ... 
29305000.0       1
58505000.0       1
1815000.0        1
58005000.0       1
6295000.0        1
Name: property_value, Length: 333, dtype: int64

In [332]:
RM.isnull().sum()

census_tract                 678
reverse_mortgage               0
open_end_line_of_credit        0
loan_amount                    0
property_value             40992
income                     36144
loan_term                    848
applicant_age                  0
lien_status                    0
tract_population               0
dtype: int64

In [333]:
RM["reverse_mortgage"].value_counts()

2    143262
1       279
Name: reverse_mortgage, dtype: int64

In [334]:
#RM = RM.dropna(subset=['loan_term','property_value','income']) """Cannot use "dropna" or "dropna subsets" without eliminating all data on reverse mortgages as RM data is a very small % of tottal data"""

In [335]:
RM["reverse_mortgage"].value_counts()

2    143262
1       279
Name: reverse_mortgage, dtype: int64

In [336]:
RM.value_counts()

census_tract  reverse_mortgage  open_end_line_of_credit  loan_amount  property_value  income  loan_term  applicant_age  lien_status  tract_population
4.802913e+10  2                 2                        165000       165000.0        40.0    360        25-34          1            7160                5
4.802917e+10  2                 2                        275000       265000.0        71.0    360        25-34          1            10857               4
4.802913e+10  2                 2                        165000       225000.0        400.0   360        35-44          1            4131                4
4.802912e+10  2                 2                        145000       235000.0        313.0   360        35-44          1            4750                4
                                                         275000       275000.0        68.0    360        25-34          1            4840                4
                                                                           

In [337]:
RM.to_csv("/content/drive/MyDrive/2020_bexar_clean.csv")

In [338]:
#Logistic Regression Model

In [339]:
lm1 = smf.ols("reverse_mortgage ~ census_tract + loan_amount + applicant_age + property_value + tract_population" , data=RM).fit()

In [340]:
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       reverse_mortgage   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     210.2
Date:                Thu, 11 Nov 2021   Prob (F-statistic):               0.00
Time:                        04:41:30   Log-Likelihood:             1.7384e+05
No. Observations:              102108   AIC:                        -3.477e+05
Df Residuals:                  102097   BIC:                        -3.476e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               1.795e-09   9.97e-11     18.000      0.000     1.6e-09    1.99e-09
applicant_age[T.35-44]  7.865e-06      0.000      0.019      0.985      -0.001       0.001
applicant_age[T.45-54]  4.954e-05      0.000      0.113      0.910      -0.001       0.001
applicant_age[T.55-64]    -0.0010      0.000     -2.047      0.041      -0.002   -4.36e-05
applicant_age[T.65-74]    -0.0137      0.001    -21.588      0.000      -0.015      -0.012
applicant_age[T.8888]     -0.0019      0.000     -3.948      0.000      -0.003      -0.001
applicant_age[T.<25]    3.087e-05      0.001      0.036      0.971      -0.002       0.002
applicant_age[T.>74]      -0.0409      0.001    -38.723      0.000      -0.043      -0.039
census_tract            4.162e-11   9.93e-15   4191.139      0.000    4.16e-11    4.16e-11
loan_amount             2.331e-09   6.58e-10      3.540      0.000    1.04e-09    3.62e-09
property_value         -1.023e-09   3.89e-10     -2.629      0.009   -1.79e-09    -2.6e-10
tract_population        9.163e-08   5.65e-08      1.623      0.105    -1.9e-08    2.02e-07
==============================================================================
Omnibus:                   229456.345   Durbin-Watson:                   0.320
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        983110566.682
Skew:                         -21.688   Prob(JB):                         0.00
Kurtosis:                     481.742   Cond. No.                     1.17e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.17e+16. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [341]:
#Train and Test Split

In [342]:
lm1_train, lm1_test = train_test_split(RM, test_size = 0.2, random_state = 20211104, 
                               shuffle=True, stratify = RM['reverse_mortgage'])

In [343]:
lm1_train['reverse_mortgage'].value_counts()

2    114609
1       223
Name: reverse_mortgage, dtype: int64

In [344]:
lm1_train['applicant_age'].value_counts()

35-44    28575
25-34    24372
45-54    21695
55-64    14574
8888     12081
65-74     7836
<25       3134
>74       2565
Name: applicant_age, dtype: int64

In [345]:
lm1_test['reverse_mortgage'].value_counts()

2    28653
1       56
Name: reverse_mortgage, dtype: int64

In [346]:
lm1_test['reverse_mortgage'].value_counts() / len(lm1_test['reverse_mortgage'])

2    0.998049
1    0.001951
Name: reverse_mortgage, dtype: float64

In [347]:
#28 Fit Model  # here is where I switch to Tableau to better exemplify the predictive aspects of this data set regardin RM's

In [348]:
#lm1 = smf.logit("reverse_mortgage ~ census_tract + loan_amount + applicant_age + property_value + tract_population", data=lm1_train).fit()
#Continuation point

In [349]:
#predict rm values using lm1_test, logit test, confusion matrix "If needed"